In [77]:
%env MODEL_NAME=densenet161
%env PROJECT_NAME=tsaikevin-inference
%env ENDPOINT=https://alpha-ml.googleapis.com/v1

env: MODEL_NAME=densenet161
env: PROJECT_NAME=tsaikevin-inference
env: ENDPOINT=https://alpha-ml.googleapis.com/v1


In [2]:
!gcloud config set project tsaikevin-inference

Updated property [core/project].


In [ ]:
!curl -X POST -v -k -H "Content-Type: application/json" -d "{'name': '"${MODEL_NAME}"'}" \
    -H "Authorization: Bearer `gcloud auth print-access-token`" \
    "${ENDPOINT}/projects/${PROJECT_NAME}/models/"

In [116]:
%env VERSION_NAME=v005

import json
import os

torchserve_version = {
  "name": "v005",
  "deployment_uri": "gs://tsaikevin-inference-server-repository/torchserve",
  "container": {
    "image": "gcr.io/tsaikevin-inference/caip-torchserve:latest",
    "args": [
    ],
    "env": [
    ], 
    "ports": [
      { "containerPort": 8080 }
    ]
  },
  "routes": {
    "predict": "/predictions/densenet161",
    "health": "/ping"
  },
  "machine_type": "n1-standard-4"
}
with open("torchserve_version.json", "w") as f: 
  json.dump(torchserve_version, f)

env: VERSION_NAME=v005


In [ ]:
!curl -X POST -v -k -H "Content-Type: application/json" -d @torchserve_version.json \
    -H "Authorization: Bearer `gcloud auth print-access-token`" \
    "${ENDPOINT}/projects/${PROJECT_NAME}/models/${MODEL_NAME}/versions"

In [120]:
!curl -X GET -k -H "Content-Type: application/json" \
    -H "Authorization: Bearer `gcloud auth print-access-token`" \
    "${ENDPOINT}/projects/${PROJECT_NAME}/models/${MODEL_NAME}/versions/${VERSION_NAME}"

{
  "name": "projects/tsaikevin-inference/models/densenet161/versions/v005",
  "isDefault": true,
  "deploymentUri": "gs://tsaikevin-inference-server-repository/torchserve",
  "createTime": "2020-05-28T09:52:03Z",
  "state": "READY",
  "etag": "vhi3slq/VM0=",
  "machineType": "n1-standard-4",
  "container": {
    "image": "gcr.io/tsaikevin-inference/caip-torchserve:latest",
    "ports": [
      {
        "containerPort": 8080
      }
    ]
  },
  "routes": {
    "predict": "/predictions/densenet161",
    "health": "/ping"
  }
}


In [121]:
!curl -O https://s3.amazonaws.com/model-server/inputs/kitten.jpg

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  108k  100  108k    0     0   148k      0 --:--:-- --:--:-- --:--:--  148k


In [123]:
!curl -X POST ${ENDPOINT}/projects/${PROJECT_NAME}/models/${MODEL_NAME}/versions/${VERSION_NAME}:predict \
    -k -H "Content-Type: application/octet-stream" \
    -H "Authorization: Bearer `gcloud auth print-access-token`" \
    --data-binary "@kitten.jpg"

[
  {
    "tiger_cat": 0.46933549642562866
  },
  {
    "tabby": 0.4633878469467163
  },
  {
    "Egyptian_cat": 0.06456148624420166
  },
  {
    "lynx": 0.0012828214094042778
  },
  {
    "plastic_bag": 0.00023323034110944718
  }
]